## Correlation

In climate data analysis we often ask questions about how two things are related to each other. 

We may have a hypothesis or physical reason why some climate phenomena may be related to or impact another part of the climate system.  We use correlation as a means to quantify this relationship. 

_Example:_

Many studies have explored the relationship between ENSO and other aspects of the climate system.  Last week, we calculated composites of precipitation based on the Nino34 index as a way of seeing how ENSO is related to changes in precipitation.  

This week we will use correlation to quantify how precipitation at each point on the globe co-varies linearly with the Nino34 index. 

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point

import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def label_latlon(ax,lons,lats):
    """ Add tick labels """
    # Define the xticks for longtitude
    ax.set_xticks(lons,crs=ccrs.PlateCarree())
    lon_formatter=cticker.LongitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)

    # Define ytick for latitude
    ax.set_yticks(lats,crs=ccrs.PlateCarree())
    lat_formatter=cticker.LatitudeFormatter()
    ax.yaxis.set_major_formatter(lat_formatter)

    return

### Read in the Nino34 index and precipitation data

In [ ]:
file_nino34='/home/pdirmeye/classes/clim680_2022/nino34_1982-2019.oisstv2_anoms.nc'
ds_nino34=xr.open_dataset(file_nino34)

file_precip='/home/pdirmeye/classes/clim680_2022/GPCP_precip.mon.mean.nc'
ds_precip = xr.open_dataset(file_precip)

### Prepare our Precipitation Data

In [ ]:
# Slice time to match Nino34 data
da_precip = ds_precip.precip.sel(time=slice(ds_nino34['time'][0],ds_nino34['time'][-1]))

da_climo = da_precip.groupby('time.month').mean()
da_anoms = da_precip.groupby('time.month')-da_climo
da_anoms

### Pick a point from our Precipitation Anomalies
EQ,180

In [ ]:
pt=da_anoms.sel(lat=0,lon=360-180,method='nearest')
pt

### Plot the Nino34 index and the point together

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(pt['time'],pt,'b')
ax2.plot(ds_nino34['time'],ds_nino34['sst'],'r')

ax1.set_ylabel('Precip. Anomaly [EQ,180˚]  $mm\;d^{-1}$', color='b')
ax2.set_ylabel('Niño3.4 Index', color='r') ;

By eye, it sort of looks like the precipitation generally goes up when Niño3.4 goes up and down when Niño3.4 goes down.  We can quantify this using `Correlation Coefficient` or also just called `Correlation`

### Correlation Coefficient

The standard formula for calculating correlation (also sometimes called Pearson's Correlation) is:

$r_{x,y}=\frac{cov(x,y)}{\sigma_x\sigma_y}$, where

$ cov(x,y)=\sum_{i=1}^{n} (x_i-\bar{x}) (y_i-\bar{y})$

It has a range of -1 to 1.
* A value of 1 means the two timeseries are perfectly correlated
* A value of -1 means they are perfectly anti-correlated
* A value of 0 means they are uncorrelated

Typically, in climate, we perform these calculations on anomalies, 
so we have already removed the means $(\bar{x},\bar{y})$, 
so our correlation equation reduces to:

$r_{x,y}=\frac{\sum_{i=1}^{n} x_i y_i} {\sigma_x\sigma_y}$

In our example:

* $x$ refers to the Niño3.4 index
* $y$ refers to our Precip anomaly at a point
* $n$ refers to the number of times (months).   

### Calculate Correlation

Calculate using the `numpy` function [corrcoef](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)

In [ ]:
R = np.corrcoef(ds_nino34['sst'],pt)
R

We expect a single number, but we get a matrix. 
Look at the documentation. 
This function returns a correlation coefficient `matrix`, meaning a matrix that is the correlation coefficients of:

$
R = 
\begin{bmatrix}
r_{xx} & r_{xy} \\
r_{yx} & r_{yy}
\end{bmatrix}
$

This can be handy if we are calcuating correlations among many time series, but here we only have two.
For our purposes, we can select the element of the correlation coefficient matrix for $r_{xy}$ or $r_{yx}$:

In [ ]:
corr = R[0,1]
corr

## Correlation over a domain

Typically, we don't want just the correlation coefficient for a single point, we want it for all points over a region. 

For example, we want to make a map so we can identify where precipitation and the Nino34 index are highly correlated. 

If we have an `xarray.DataArray` we can use the `xarray.corr` function.

In [ ]:
xr.corr?

In [ ]:
r_map = xr.corr(ds_nino34['sst'],da_anoms,dim='time')
r_map

We get back an `xarray.DataArray` corresponding to a map of correlation coefficients. 

### Plot our correlation map

In [ ]:
clevs=np.arange(-1,1.1,0.1)

fig = plt.figure(figsize=(11,8.5))

# Set the axes using the specified map projection
ax=plt.axes(projection=ccrs.PlateCarree(central_longitude=200))

# Add cyclic point
data=r_map
data,lon=add_cyclic_point(r_map,coord=da_anoms['lon'])

# Make a filled contour plot
cs=ax.contourf(lon,da_anoms['lat'],
            data,clevs,
            transform=ccrs.PlateCarree(),
            cmap='seismic',extend='both')

# Add coastlines
ax.coastlines()

# Add gridlines
ax.gridlines()

label_latlon(ax,np.arange(-180,181,60),np.arange(-90,91,30))
# Define the xticks for longtitude 
#ax.set_xticks(np.arange(-180,181,60),crs=ccrs.PlateCarree())
#lon_formatter=cticker.LongitudeFormatter()
#ax.xaxis.set_major_formatter(lon_formatter)

# Define ytick for latitude
#ax.set_yticks(np.arange(-90,91,30),crs=ccrs.PlateCarree())
#lat_formatter=cticker.LatitudeFormatter()
#ax.yaxis.set_major_formatter(lat_formatter)

# Call colorbar
cbar=plt.colorbar(cs,orientation='horizontal',shrink=0.7,
                 label='Correlation Coefficient')

# Add title
plt.title('Correlation between Nino3.4 and Precipitation Anomalies') ;

### Interpreting our Correlation

* Red areas indicate that the precipitation goes up with Niño3.4 goes up and down when Nino34 goes down.
* Blue area indicate the oppposite: precipitation increases when Niño3.4 decreases and decreases when Niño3.4 increases. 
* The values near zero indicate no relationship between precipitation anomalies and Niño3.4. 

### How do we determine how close to zero means there is no relationship vs. a relationship?

Calculate statistical significance

## Important Reminder

You have probably heard the phrase `Correlation is NOT causation`, but what does this mean?

It means that just because two variables are highly correlated, does not mean that one variable causes the other one to vary in a certain way.  

Attributing cause comes from our understanding of the physical climate system. We have a physical understanding of how ENSO impacts the atmospheric circulation and therefore precipitation. We use correlation as a way to quantify that relationship.  

**Always be suspicious of a correlation that makes no sense!** 

-------------
## Statistical Significance of a Correlation

The purpose of statistical significance is to ensure we don't make conclusions about results obtained by random chance. 
We want to be sure that when we find a correlation, there is in fact a likely relationship.

Let's calculate statistical significance of the correlation using the `stats` package and another correlation function: `pearsonr`.

This function calculates both the correlation coefficient and the p-value indicating it the % chance that we could be wrong about it being different from zero.

First, let's look at the help for `pearsonr`


In [ ]:
from scipy.stats import pearsonr
pearsonr?

This will not work, why not: 
r,p=pearsonr(ds_nino34['sst'],ds_anoms['precip'])

In [ ]:
nx=len(da_anoms['lon'])
ny=len(da_anoms['lat'])

p_array=np.zeros((ny,nx))
r_array=np.zeros((ny,nx))

t_start = time.perf_counter()
for i in range(nx):
    for j in range(ny):
        r,p = pearsonr(ds_nino34['sst'],da_anoms[:,j,i])
        r_array[j,i] = r
        p_array[j,i] = p
#r_array.shape
print(f"{time.perf_counter()-t_start:0.2f} s")

### Let's take a peek

In [ ]:
plt.contourf(r_array,cmap='seismic')
plt.title("Correlations")
plt.colorbar() ;

In [ ]:
plt.contourf(p_array,[0,0.01,0.05,0.1],cmap='Greys_r')
plt.title("p-Values")
plt.colorbar() ;

### Plot our correlation including significance

A p-value of 0.05 corresponds to a 5% probability a correlation at that location arose by chance, or to a 95% confidence level.

In [ ]:
mask_sig=np.where(p_array<0.05,r_array,np.nan)

In [ ]:
clevs=np.arange(-1,1.1,0.1)

fig = plt.figure(figsize=(11,8.5))

# Set the axes using the specified map projection
ax=plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

# Add cyclic point
data=r_array
data,lon=add_cyclic_point(data,coord=da_anoms['lon'])
mask_data,lons=add_cyclic_point(mask_sig,coord=da_anoms['lon'])

# Make a filled contour plot
cs=ax.contourf(lon,da_anoms['lat'],
            data,clevs,
            transform=ccrs.PlateCarree(),
            cmap='bwr',extend='both')

ax.contourf(lon,da_anoms['lat'],mask_data,[0,1],
            transform = ccrs.PlateCarree(),colors='None',
            hatches=['//','\\\\'],extend='both',alpha=0)

# Add coastlines
ax.coastlines()

# Add gridlines
ax.gridlines()

# Define the xticks for longtitude 
ax.set_xticks(np.arange(-180,181,60),crs=ccrs.PlateCarree())
lon_formatter=cticker.LongitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)

# Define ytick for latitude
ax.set_yticks(np.arange(-90,91,30),crs=ccrs.PlateCarree())
lat_formatter=cticker.LatitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)

# Call colorbar
cbar=plt.colorbar(cs,orientation='horizontal',shrink=0.7,
                 label='Correlation Coefficient')

# Add title
plt.title('Correlation between Niño3.4 and Precipitation Anomalies') ;

### A Note about Field Significance and False Discovery

When looking at a map of correlation, or any statistic, 
where statistical significance is indicated, keep in mind the definition of significance.

At a threshold of $p=0.05$ (95% confidence level), 
we would expect 5% of locations to appear significant
even if the two fields we are correlating are nothing but random noise.
How do we account for this?

#### 1. Calculate significant area

A simple way is to calculate the fraction of the domain that registers as significant. 
Remember, we generated a mask array `mask_sig` that could be used, summing the areas of grid cells
containing valid numbers (remember to weight by `np.cos(lat)`) 
divided by the sum of all grid cells, also weighted appropriately.

#### 2. Control for false discovery

There is another way. One can adjust the calculated values of $p$ based on their ranked distribution.  
Applied to p values from a correlation between truly random fields, random significance would be removed.
For correlations that have genuine field significance, the p values shift up slightly to adjust for 
the impact of random apparent signficance.

Paper discussing the need to "control for the false discovery rate" - https://journals.ametsoc.org/view/journals/bams/97/12/bams-d-15-00267.1.xml 

Blog post explaining thresholding with false discovery rate https://matthew-brett.github.io/teaching/fdr.html

Python package with fdr correction (requires Python >= 3.7)
https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.fdrcorrection.html

In [ ]:
from scipy.stats import rankdata

def fdr(p_vals):
    # from https://stackoverflow.com/questions/25185205/calculating-adjusted-p-values-in-python  
    ranked_p_values = rankdata(p_vals)
    fdr = p_vals * len(p_vals) / ranked_p_values
    fdr[fdr > 1] = 1

    return fdr

In [ ]:
# function expecting vector so reshape 2D p value array as vector
p_array_vec=np.ravel(p_array, order='C')
p_array_corrected_vec=fdr(p_array_vec)
# reshape corrected p value vectore back to 2D array
p_array_corrected=np.reshape(p_array_corrected_vec, (72,144), order='C')

In [ ]:
plt.plot(p_array_corrected_vec[5170:5199],'*')
plt.plot(p_array_vec[5170:5199],'x') 
plt.legend(['Corrected p','Original p'])
plt.title("False Discovery Correction\nAt a Sample of Grid Cells") ;

In [ ]:
mask_sig2=np.where(p_array_corrected<0.05,r_array,np.nan)

clevs=np.arange(-1,1.1,0.1)

fig = plt.figure(figsize=(11,8.5))

# Set the axes using the specified map projection
ax=plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

# Add cyclic point
data=r_array
data,lon=add_cyclic_point(data,coord=da_anoms['lon'])
mask_data,lons=add_cyclic_point(mask_sig2,coord=da_anoms['lon'])

# Make a filled contour plot
cs=ax.contourf(lon,da_anoms['lat'],
            data,clevs,
            transform=ccrs.PlateCarree(),
            cmap='bwr',extend='both')

ax.contourf(lon,da_anoms['lat'],mask_data,[0,1],
            transform = ccrs.PlateCarree(),colors='None',
            hatches=['//','\\\\'],extend='both',alpha=0)

# Add coastlines
ax.coastlines()

# Add gridlines
ax.gridlines()

# Define the xticks for longtitude 
ax.set_xticks(np.arange(-180,181,60),crs=ccrs.PlateCarree())
lon_formatter=cticker.LongitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)

# Define ytick for latitude
ax.set_yticks(np.arange(-90,91,30),crs=ccrs.PlateCarree())
lat_formatter=cticker.LatitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)

# Call colorbar
cbar=plt.colorbar(cs,orientation='horizontal',shrink=0.7,
                 label='Correlation Coefficient')

# Add title
plt.title('Correlation between Nino3.4 and Precipitation Anomalies')